# Identificando Entidades Nomeadas
Neste capítulo vamos treinar um identificador de Entidades (NER) nomeadas usando a biblioteca [Spacy](https://spacy.io). A partir deste capítulo vamos importar também funções que já criamos anteriormente, e que encontram-se reproduzidas em  [dhbbmining.py](dhbbmining.py)

In [38]:
import os, glob
import spacy
from spacy import displacy
from sqlalchemy import create_engine
from dhbbmining import *
import ipywidgets as widgets

Para utilizar o spacy em um corpus na lingua portuguesa, vamos primeiro importar o modelo liguístico do português

In [19]:
nlp = spacy.load("pt_core_news_sm")

Em seguida podemos carregar os verbetes bográficos que salvamos no nosso banco SQLite.

In [17]:
eng = create_engine("sqlite:///minha_tabela.sqlite")
#dhbb = pd.read_sql_table('resultados')
biograficos = pd.read_sql_query('select * from resultados where natureza="biográfico"', con=eng)
biograficos.head()

,index,arquivo,title,natureza,sexo,cargos,corpo
0,0,1.text,"COELHO, Machado",biográfico,m,\n - dep. fed. DF 1927-1929 \n - dep. fed. DF ...,\n\n«José Machado Coelho de Castro» nasceu em ...
1,1,10.text,"ABÍLIO, Armando",biográfico,m,\n - dep. fed. PB 1995-1999\n - dep. fed. PB ...,\n\n«Armando Abílio Vieira» nasceu em Itaporan...
2,2,100.text,"ALEIXO, Pedro",biográfico,m,\n - const. 1934\n - dep. fed. MG 1935-1937\n ...,"\n\n«Pedro Aleixo» nasceu em São Caetano, dist..."
3,3,1000.text,"CAMPOS, Eduardo",biográfico,m,\n - dep. fed. PE 1995\n - dep. fed. PE 1998-...,\n\n«Eduardo Henrique Accioly Campos» nasceu e...
4,4,1001.text,"CAMPOS, Eleazar Soares",biográfico,m,\n - magistrado\n - interv. MA 1945-1946\n,\n\n«Eleazar Soares Campos» nasceu em São Luís...


Para começar a utilizar o Spacy, precisamos primeiro precisamos processar o texto. Nesta passagem várias análises linguísticas são realizadas.

In [35]:
doc = nlp(biograficos.corpo[0].strip())
type(doc)

spacy.tokens.doc.Doc

In [36]:
for i, token in enumerate(doc):
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    if i>5:
        break
    

« « PUNCT PU|@PU punct « False False
José José PROPN PROPN nsubj Xxxx True False
Machado Machado PROPN PROPN flat:name Xxxxx True False
Coelho Coelho PROPN PROPN flat:name Xxxxx True False
de de ADP PRP|@N< case xx True True
Castro Castro PROPN PROP|@P< nmod Xxxxx True False
» » PUNCT PU|@PU punct » False False


## Testando o NER do Spacy
Como o Spacy já inclui algum suporte à lingua portuguesa, antes de pensar em treinar nosso próprio NER, podemos avaliar a performance do existente. 

Abaixo vamos construir uma visualização interativa da marcação de entidades nos verbetes do DHBB.

In [63]:
from IPython.display import display,clear_output
from ipywidgets import interact

In [73]:
@interact(e=(0, len(biograficos)))
def mostra_ner(e=0):
    text = biograficos.iloc[e].corpo.strip()
    doc = nlp(text)
    displacy.render(doc, style="ent", jupyter=True)
    clear_output(wait=True)

interactive(children=(IntSlider(value=0, description='e', max=6724), Output()), _dom_classes=('widget-interact…

Além da visualização, podemos extrair as entidades presentes em um verbete:

In [74]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

José Machado Coelho de Castro» 1 31 PER
Lorena 42 48 LOC
SP 50 52 LOC
Ginásio Diocesano de São Paulo 67 97 LOC
Faculdade de Ciências Jurídicas 127 158 ORG
Sociais 161 168 LOC
Cunha 220 225 LOC
SP 227 229 LOC
Rio de
Janeiro 263 277 LOC
Distrito Federal 285 301 LOC
Distrito Federal 357 373 LOC
Câmara 488 494 LOC
Getúlio Vargas 575 589 PER
Ligado 591 597 LOC
Washington Luís 654 669 PER
Guanabara 682 691 LOC
Alcântara 951 960 LOC
Lisboa 965 971 LOC
Getúlio 1050 1057 PER
Vargas 1058 1064 PER
São 1125 1128 LOC
Paulo 1129 1134 LOC
São 1206 1209 LOC
Paulo 1210 1215 LOC
Partido Social Democrático 1231 1257 ORG
PSD 1259 1262 ORG
Assembléia Nacional Constituinte 1284 1316 ORG
ANC 1318 1321 ORG
Carta
 1484 1490 PER
Constituinte 1523 1535 MISC
Congresso 1539 1548 LOC
Comissão Permanente de Obras Públicas da Câmara Federal 1571 1626 ORG
Câmara 1732 1738 ORG
Companhia de Cimento Vale do Paraíba 1784 1820 ORG
Rio de Janeiro 1834 1848 LOC


## Treinando Um identificador de Entidades a partir do DHBB
Identificadores de entidades são algoritmos treinados em corpora manualmente anotados. Como cada corpora possui um conjunto particular de entidades, para uma performance ótima o ideal é treinarmos o modelo no Próprio DHBB. Para este fim utilizaremos os dicionários já disponíveis no DHBB,  juntamente com o índice construído no capítulo 2 para recuperar o contexto de cada entrada dos dicionários.

In [2]:
from whoosh import index 
import os
from whoosh.qparser import QueryParser
from whoosh import qparser

O primeiro passo é abrirmos o nosso indice.

In [3]:
if os.path.exists('indexdir'):
    indice = index.open_dir('indexdir')

In [5]:
indice.doc_count

6724

In [10]:
def busca(consulta):
    qp = QueryParser("corpo", indice.schema)
    qp.add_plugin(qparser.EveryPlugin())
    query = qp.parse(consulta)
    
    with indice.searcher() as searcher:
        results = [(dict(hit),hit.highlights("corpo")) for hit in searcher.search(query, limit=None)]
    return results

In [12]:
resultados = busca('"filho de"')[0]

({'caminho': '3822.text',
  'corpo': '\n\n«Paulo Nogueira Filho» nasceu na cidade de São Paulo no dia 16 de\nnovembro de 1898, filho de Paulo de Almeida Nogueira e de Ester\nNogueira. Seu avô, José Paulino Nogueira, foi coronel da Guarda\nNacional, fazendeiro, comerciante e iniciador de grandes empreendimentos\nfinanceiros.\n\nDescendente de famílias abastadas, proprietárias de fazendas e\nindústrias, Paulo Nogueira Filho conviveu desde a infância com\ndestacadas personalidades do mundo econômico e político da época, como\nManuel de Campos Sales, José Gomes Pinheiro Machado, Carlos Guimarães,\nJosé Maria Whitaker e outros. Recebeu educação cuidadosa, freqüentando\ndurante dois anos uma escola na Europa. Concluiu os estudos secundários\nno Colégio São Bento, na capital paulista, e, depois de estudar durante\nsete meses no curso preparatório Henrique Greenen, ingressou em 1914 na\nFaculdade de Direito de São Paulo. Desde o início do curso, desenvolveu\nintensa atividade política, iniciad

Agora já temos os ingredientes necessários para treinar um modelo de entidades usando a biblioteca spacy.

In [13]:
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

Primeiro precisamos criar o conjunto de treinamento do modelo. e deve ter  a forma de uma lista como a descrita abaixo.
```python
    TRAIN_DATA = [
        ("nasceu em Itaporanga ( PB ) no dia 29 de dezembro de 1944 , filho de Argemiro Abílio de Sousa", 
         {"entities": [(10, 20, "LOC"), (69, 93, "PERSON")]}
        ),
    ]
```

In [14]:
def gera_dados_treinamento(dicionário, tag):
    pass

69

In [ ]:
def main(model=pt, output_dir=dhbb_nlp, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("pt")  # create blank Language class
        print("Created blank 'pt' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])